In [ ]:
class NoScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, x, y=None):
        return self

    def transform(self, x, y=None):
        return x

class Classifier:
    def __init__(self, params):
        self.params = params
        if params['standardize'] == 'StandardScaler':
            self.standardizer = StandardScaler()
        elif params['standardize'] == 'MinMaxScaler':
            self.standardizer = MinMaxScaler()
        elif params['standardize'] == 'NoScaler':
            self.standardizer = NoScaler()

        if params['classifier_name'] == 'RandomForest':
            self.classifier = RandomForestClassifier(**params['classifier_params'])
        elif params['classifier_name'] == 'SVC':
            self.classifier = SVC(**params['classifier_params'])

    def _fit_and_predict_core(self, x, y=None, fitting=False, proba=False):
        if fitting == True:
            self.standardizer.fit(x)
        self.standardizer.transform(x)

        if fitting == True:
            self.classifier.fit(x, y)
        if y is None:
            if proba:
                return self.classifier.predict_proba(x)
            else:
                return self.classifier.predict(x)
        return None

    def fit(self, x, y):
        self._fit_and_predict_core(x, y, fitting=True)
        return self

    def predict(self, x):
        pred_y = self._fit_and_predict_core(x)
        return pred_y

    def predict_proba(self, x):
        pred_y = self._fit_and_predict_core(x, proba=True)
        return pred_y


In [ ]:
class Objective:
    def __init__(self, x, y, label_index):
        self.x = x
        self.y = y
        self.label_index = label_index
        self.best_score = 0
        self.best_params = None

    def __call__(self, trial):
        x = self.x
        y = self.y
        params = self.generate_params(trial, x)

        classifier = Classifier(params)

        skf = StratifiedKFold(n_splits=5, random_state=2019, shuffle=True)

        ts = []
        ys = []

        for train, test in skf.split(x, y):
            train_y = y.iloc[train].values
            test_y = y.iloc[test].values
            train_x = x.iloc[train].values
            test_x = x.iloc[test].values

            classifier.fit(train_x, train_y)
            pred_y = classifier.predict(test_x).reshape(-1).astype(np.int)
            ts.append(test_y)
            ys.append(pred_y)

        y_true = self.label_index[np.concatenate(ts)]
        y_pred = self.label_index[np.concatenate(ys)]

        f1_score = metrics.f1_score(y_true, y_pred, average='macro')
        if self.best_score < f1_score:
            self.best_score = f1_score
            self.best_params = params
        return f1_score

    def generate_params(self, trial, x):
        params = {}

        params['standardize'] = trial.suggest_categorical('standardize', ['NoScaler', 'StandardScaler', 'MinMaxScaler'])
        params['classifier_name'] = trial.suggest_categorical('classifier_name', ['RandomForest', 'SVC'])

        classifier_params = {}
        if params['classifier_name'] == 'SVC':
            classifier_params['kernel'] = trial.suggest_categorical('svc_kernel',
                                                             ['linear', 'rbf'])
            classifier_params['C'] = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
            if classifier_params['kernel'] == 'rbf':
                classifier_params['gamma'] = trial.suggest_categorical('svc_gamma',
                                                           ['auto', 'scale'])
            else:
                classifier_params['gamma'] = 'auto'

        elif params['classifier_name'] == 'RandomForest':
            classifier_params['n_estimators'] = trial.suggest_categorical(
                'rf_n_estimators', [5, 10, 20, 30, 50, 100])
            classifier_params['max_features'] = trial.suggest_categorical(
                'rf_max_features', ['auto', 0.2, 0.4, 0.6, 0.8])
            classifier_params['max_depth'] = int(
                trial.suggest_loguniform('rf_max_depth', 2, 32))
            classifier_params['n_jobs'] = -1
        else:
            raise RuntimeError('unspport classifier', params['classifier_name'])
        params['classifier_params'] = classifier_params
        return params


In [ ]:
import numpy as np
import pandas as pd
import optuna
from sklearn import datasets, metrics
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin

wine = datasets.load_wine()
wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_label = pd.Series(wine.target)
label, label_index = pd.factorize(wine_label)


In [ ]:
wine_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [1]:
objective = Objective(x=wine_df, y=wine_label, label_index=label_index)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)


NameError: ignored

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=44, shuffle=True)
x = wine_df
y = wine_label

print(study.best_trial)
best_score = objective.best_score
print('best score:', best_score)
best_params = objective.best_params
print('best params:', best_params)

ts = []
ys = []

for train, test in skf.split(x, y):
    train_y = y.iloc[train].values
    test_y = y.iloc[test].values
    train_x = x.iloc[train]
    test_x = x.iloc[test]
    model = Classifier(best_params)
    model.fit(train_x, train_y)
    predict = model.predict(test_x).reshape(-1).astype(np.int)
    ts.append(test_y)
    ys.append(predict)
y_true = label_index[np.concatenate(ts)]
y_pred = label_index[np.concatenate(ys)]
print(metrics.confusion_matrix(y_true=y_true, y_pred=y_pred))
print(metrics.classification_report(y_true=y_true, y_pred=y_pred))


FrozenTrial(number=3, value=0.9890005486730775, datetime_start=datetime.datetime(2020, 7, 12, 21, 26, 31, 798060), datetime_complete=datetime.datetime(2020, 7, 12, 21, 26, 33, 619228), params={'standardize': 'StandardScaler', 'classifier_name': 'RandomForest', 'rf_n_estimators': 100, 'rf_max_features': 0.2, 'rf_max_depth': 3.3004885989708286}, distributions={'standardize': CategoricalDistribution(choices=('NoScaler', 'StandardScaler', 'MinMaxScaler')), 'classifier_name': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'rf_n_estimators': CategoricalDistribution(choices=(5, 10, 20, 30, 50, 100)), 'rf_max_features': CategoricalDistribution(choices=('auto', 0.2, 0.4, 0.6, 0.8)), 'rf_max_depth': LogUniformDistribution(high=32, low=2)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE)
best score: 0.9890005486730775
best params: {'standardize': 'StandardScaler', 'classifier_name': 'RandomForest', 'classifier_params': {'n_estimators': 100

ベーシック

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_uniform("x", -10, 10)
    score = (x - 2) ** 2
    print("x: %1.3f  score: %1.3f"%(x,score))
    return score

study = optuna.create_study()
study.optimize(objective, n_trials=100)

x: 9.542  score: 56.880


[I 2020-07-12 22:06:22,838] Finished trial#0 with value: 56.87954610992066 with parameters: {'x': 9.54185296262932}. Best is trial#0 with value: 56.87954610992066.


x: 6.099  score: 16.799


[I 2020-07-12 22:06:22,951] Finished trial#1 with value: 16.799167372847005 with parameters: {'x': 6.098678735012907}. Best is trial#1 with value: 16.799167372847005.


x: 8.739  score: 45.409


...

[I 2020-07-12 22:06:33,590] Finished trial#97 with value: 0.6579819531100228 with parameters: {'x': 2.8111608675904076}. Best is trial#58 with value: 0.00012449281556505207.


x: 1.382  score: 0.382


[I 2020-07-12 22:06:33,699] Finished trial#98 with value: 0.3815561373594586 with parameters: {'x': 1.3822976951965789}. Best is trial#58 with value: 0.00012449281556505207.


x: 0.179  score: 3.315


[I 2020-07-12 22:06:33,810] Finished trial#99 with value: 3.3154855842271083 with parameters: {'x': 0.17915250934431404}. Best is trial#58 with value: 0.00012449281556505207.


In [ ]:
study.best_params

{'x': 2.011157634855338}